In [73]:
import os

# TEMP CREDENTIALS


ACCOUNT_ID = 'xxxxxx'


In [74]:
import boto3
import pandas as pd
import json
import datetime
import os
client = None
client = boto3.client("quicksight", region_name="ap-southeast-2")
#client = session.client("quicksight", region_name="ap-southeast-2")

# Analyse Quicksight Assets in an account

## Looking at the following (excludes new items like Stories)
- Data Sources
- Data Sets
- Analysis
- Dashboards
- Old Users within QS (not federated)
- Old Groups within QS



## Lets start with Data Sources

In [84]:
# Get list of data sources from AWS QuickSight
list_data_sources_response = client.list_data_sources(
    AwsAccountId=ACCOUNT_ID,
)
list_data_sources_response

# Create pandas DataFrame from data sources response
data_source_df = pd.DataFrame(list_data_sources_response["DataSources"])

# Convert datetime columns to date format
date_columns = data_source_df.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    data_source_df[date_column] = data_source_df[date_column].dt.date
    
# Initialize empty list to store data source permissions
data_source_permission_list = []

# Iterate through data sources and get permissions for each
for index, row in data_source_df.iterrows():
    data_source_id = row["DataSourceId"]
    data_source_name = row["Name"]

    # Get permissions for current data source
    describe_data_sources_perm_response = client.describe_data_source_permissions(
        AwsAccountId=ACCOUNT_ID,
        DataSourceId=data_source_id
    )

    # Extract permission details and add to list
    for permission_entry in describe_data_sources_perm_response["Permissions"]:
        data_source_permission_list.append({
                "DataSourceId": data_source_id,
                "DataSourceName": data_source_name,
                "Principal": permission_entry["Principal"],
                "Actions": permission_entry["Actions"]
            })

# Create DataFrame from permissions list
data_source_permission_df = pd.DataFrame(data_source_permission_list)


## Now the Data Sets

In [90]:
# Get list of all datasets in the account
list_data_sets_response = client.list_data_sets(
    AwsAccountId=ACCOUNT_ID,
)

# Convert dataset list to pandas DataFrame
data_set_df = pd.DataFrame(list_data_sets_response["DataSetSummaries"])

# Convert datetime columns to date format
date_columns = data_set_df.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    data_set_df[date_column] = data_set_df[date_column].dt.date

# Initialize lists to store datasource and permission info
dataset_datasource_list = []
dataset_permission_list = []

# Iterate through each dataset to get details
for index, row in data_set_df.iterrows():
    data_set_id = row["DataSetId"]
    data_set_name = row["Name"]

    # Get detailed information about the dataset
    try:
        data_set_describe_response = client.describe_data_set(
            AwsAccountId=ACCOUNT_ID,
            DataSetId=data_set_id
        )
        
        # Extract datasource information from physical table mappings
        for source in data_set_describe_response["DataSet"]["PhysicalTableMap"]:
            for inner_source, details in data_set_describe_response["DataSet"]["PhysicalTableMap"][source].items():
                dataset_datasource_list.append({
                    "DataSetId": data_set_id,
                    "DataSourceArn": details["DataSourceArn"]
                })
    except Exception as e:
        print(f"Error getting details for dataset {data_set_id}: SKIPPING Data Source for this - MOST LIKELY ITS CSV / FILE UPLOAD ?")


    # Get permissions associated with the dataset
    describe_data_set_perm_response = client.describe_data_set_permissions(
        AwsAccountId=ACCOUNT_ID,
        DataSetId=data_set_id
    )

    # Extract permission details
    for permission_entry in describe_data_set_perm_response["Permissions"]:
        dataset_permission_list.append({
                "DataSetId": data_set_id,
                "DataSetName": data_set_name,
                "Principal": permission_entry["Principal"],
                "Actions": permission_entry["Actions"]
            })
        
# Convert datasource list to DataFrame
dataset_datasource_df = pd.DataFrame(dataset_datasource_list)

# Convert permissions list to DataFrame
dataset_permission_df = pd.DataFrame(dataset_permission_list)



Error getting details for dataset 30fae72e-0a7a-4ef9-bcae-ad35511f04c6: SKIPPING Data Source for this - MOST LIKELY ITS CSV / FILE UPLOAD ?


## Analyses

In [91]:
# Get list of all analyses for the account
list_analyses_response = client.list_analyses(
    AwsAccountId=ACCOUNT_ID,
)
# Convert analyses list to pandas dataframe
analyses_df = pd.DataFrame(list_analyses_response["AnalysisSummaryList"])

# Initialize empty lists to store datasources and permissions
analysis_datasource_list = []
analysis_permission_list = []

# Convert datetime columns to date format
date_columns = analyses_df.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    analyses_df[date_column] = analyses_df[date_column].dt.date

# Iterate through each analysis to get datasources and permissions
for index, row in analyses_df.iterrows():

    analyses_id = row["AnalysisId"]
    analysis_name = row["Name"]

    # Get detailed information about the analysis
    analyses_describe_response = client.describe_analysis(
        AwsAccountId=ACCOUNT_ID,
        AnalysisId=analyses_id
    )

    # Extract and store datasource information
    for dataset in analyses_describe_response["Analysis"]["DataSetArns"]:
        analysis_datasource_list.append(
            {'AnalysisId': analyses_id,
            'DataSetArn': dataset
            }
        )
    
    # Get permissions for the analysis
    describe_analysis_perm_response = client.describe_analysis_permissions(
        AwsAccountId=ACCOUNT_ID,
        AnalysisId=analyses_id
    )

    # Extract and store permission information
    for permission_entry in describe_data_set_perm_response["Permissions"]:
        analysis_permission_list.append({
                "AnalysisId": analyses_id,
                "AnalysisName": analysis_name,
                "Principal": permission_entry["Principal"],
                "Actions": permission_entry["Actions"]
            })
        
# Convert datasource and permission lists to dataframes
analysis_datasource_df = pd.DataFrame(analysis_datasource_list)
analysis_permission_df = pd.DataFrame(dataset_permission_list)


## Dashboards

In [92]:
# Get list of all dashboards for the account
list_dashboards_response = client.list_dashboards(
    AwsAccountId=ACCOUNT_ID,
)
# Convert dashboard list to pandas dataframe
dashboard_df = pd.DataFrame(list_dashboards_response["DashboardSummaryList"])

# Convert datetime columns to date format
date_columns = dashboard_df.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    dashboard_df[date_column] = dashboard_df[date_column].dt.date

# Initialize lists to store dashboard analysis and permissions
dashboard_analysis_list = []
dashboard_permission_list = []

# Iterate through each dashboard to get additional details
for index, row in dashboard_df.iterrows():
    dashboard_id = row["DashboardId"]
    
    # Get detailed dashboard information
    dashboard_describe_response = client.describe_dashboard(
        AwsAccountId=ACCOUNT_ID,
        DashboardId=dashboard_id
    )

    # Extract and store analysis ARN for the dashboard
    dashboard_analysis_list.append(
            {'DashboardId': dashboard_id,
            'AnalysisArn': dashboard_describe_response["Dashboard"]["Version"]["SourceEntityArn"]
            }
    )

    # Get dashboard permissions
    describe_dashboard_perm_response = client.describe_dashboard_permissions(
        AwsAccountId=ACCOUNT_ID,
        DashboardId=dashboard_id
    )

    # Extract and store permission details for each principal
    for permission_entry in describe_dashboard_perm_response["Permissions"]:
        dashboard_permission_list.append({
                "DashboardId": dashboard_id,
                "Principal": permission_entry["Principal"],
                "Actions": permission_entry["Actions"]
            })

# Convert analysis and permissions lists to pandas dataframes
dashboard_analysis_df = pd.DataFrame(dashboard_analysis_list)
dashboard_permission_df = pd.DataFrame(dashboard_permission_list)



## Quicksight Users

In [93]:
# Call QuickSight API to get list of users in the account
list_users_response = client.list_users(
    AwsAccountId=ACCOUNT_ID,
    Namespace='default'
)

# Convert the user list response to a pandas DataFrame for easier manipulation
users_df = pd.DataFrame(list_users_response["UserList"])


## Quicksight Groups and Memberships

In [94]:
# Get list of all groups in the default namespace
list_groups_response = client.list_groups(
    AwsAccountId=ACCOUNT_ID,
    Namespace='default'
)

# Convert groups list to pandas DataFrame
groups_df = pd.DataFrame(list_groups_response["GroupList"])

# Initialize empty list to store group membership details
group_membership_list = []

# Iterate through each group
for index, row in groups_df.iterrows():
    group_name = row["GroupName"]
    
    # Get list of members for current group
    group_membership_response = client.list_group_memberships(
        AwsAccountId=ACCOUNT_ID,
        GroupName=group_name,
        Namespace='default' )
    
    
    # For each member in the group, add their details to the list
    for group_membership_entry in group_membership_response["GroupMemberList"]:
        group_membership_list.append(
            {'GroupName': group_name,
             'MemberArn': group_membership_entry["Arn"],
            'MemberName': group_membership_entry["MemberName"]
            }
        )

# Convert membership list to pandas DataFrame
group_membership_df = pd.DataFrame(group_membership_list)


## Create an excel file with outputs

In [97]:
# Specify the path to the Excel file
file_path = 'quicksight_analysis.xlsx'

# Write each DataFrame to a different sheet in the Excel file
with pd.ExcelWriter(file_path) as writer:
    data_source_df.to_excel(writer, sheet_name='DataSources', index=False)
    data_source_permission_df.to_excel(writer, sheet_name='DataSourcePermissions', index=False)
    data_set_df.to_excel(writer, sheet_name='DataSets', index=False)
    dataset_datasource_df.to_excel(writer, sheet_name='DataSet_DataSource', index=False)
    dataset_permission_df.to_excel(writer, sheet_name='DataSetPermissions', index=False)
    analyses_df.to_excel(writer, sheet_name='Analyses', index=False)
    analysis_datasource_df.to_excel(writer, sheet_name='Analysis_DataSource', index=False)
    analysis_permission_df.to_excel(writer, sheet_name='AnalysisPermissions', index=False)
    dashboard_df.to_excel(writer, sheet_name='Dashboards', index=False)
    dashboard_analysis_df.to_excel(writer, sheet_name='Dashboard_Analysis', index=False)
    dashboard_permission_df.to_excel(writer, sheet_name='DashboardPermissions', index=False)
    users_df.to_excel(writer, sheet_name='Users', index=False)
    groups_df.to_excel(writer, sheet_name='Groups', index=False)
    group_membership_df.to_excel(writer, sheet_name='GroupMembership', index=False)

print(f"Excel file created at {file_path}")

Excel file created at quicksight_analysis.xlsx
